In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.launching.run_super import *
from amftrack.pipeline.launching.run import *

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Experiment,
    save_graphs,
)

In [14]:
directory = r"/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince2/Images/"
update_plate_info(directory, local=True)
all_folders = get_current_folders(directory, local=True)

analysed:   0%|          | 0/236 [00:00<?, ?it/s]

In [23]:
folders = all_folders.loc[all_folders["PrincePos"] == "4"]
folders = folders.sort_values("folder", ascending=False).iloc[:1]

In [24]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
213,/run/user/357100554/gvfs/smb-share:server=prin...,429,4,'Carrot','A5','001P100N100C','Y','-',20220610,25,...,False,False,False,False,False,False,"14.10.2022, 10:07:",20221014_1007_Plate04,429_20220610,2022-10-14 10:07:00


In [25]:
path_code = os.getenv("HOME") + "/pycode/MscThesis/"


def make_stitching_loop(directory, dirname, op_id):
    a_file = open(
        f"{path_code}amftrack/pipeline/scripts/stitching_loops/stitching_loop2.ijm", "r"
    )

    list_of_lines = a_file.readlines()

    list_of_lines[4] = f"mainDirectory = \u0022{directory}\u0022 ;\n"
    list_of_lines[29] = f"\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n"
    file_name = f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm'
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()


def run_stitch(directory, folders):
    folder_list = list(folders["folder"])
    folder_list.sort()
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for folder in folder_list:
            op_id = time_ns()
            im = imageio.imread(f"{directory}/{folder}/Img/Img_r03_c05.tif")
            for x in range(1, 11):
                for y in range(1, 16):
                    strix = str(x) if x >= 10 else f"0{x}"
                    striy = str(y) if y >= 10 else f"0{y}"
                    path = f"{directory}/{folder}/Img/Img_r{strix}_c{striy}.tif"
                    # print(striy,path,os.path.getsize(path))
                    if not os.path.isfile(path):
                        f = open(path, "w")
                    if os.path.getsize(path) == 0:
                        imageio.imwrite(path, im * 0)
            make_stitching_loop(directory, folder, op_id)
            command = [
                os.getenv("HOME") + "/Fiji.app/ImageJ-linux64",
                "--mem=8000m",
                "--headless",
                "--ij2",
                "--console",
                "-macro",
                f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm',
            ]
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)

In [26]:
run_stitch(directory, folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

/data/temp/ipykernel_2559802/1705550735.py:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imageio.imread(f"{directory}/{folder}/Img/Img_r03_c05.tif")


/home/ipausers/bisot/Fiji.app/ImageJ-linux64 --mem=8000m --headless --ij2 --console -macro /data/temp/stitching_loops/stitching_loop1665742933784116528.ijm


Java HotSpot(TM) 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
Java HotSpot(TM) 64-Bit Server VM warning: Using incremental CMS is deprecated and will likely be removed in a future release


20221014_1007_Plate04/
(Fri Oct 14 12:24:52 CEST 2022): Stitching the following files:
Image: '/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince2/Images//20221014_1007_Plate04//Img2/Img_r01_c01.tif' Imp: 'null' Offset: (0.0, 0.0)
Image: '/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince2/Images//20221014_1007_Plate04//Img2/Img_r01_c02.tif' Imp: 'null' Offset: (3276.0, 0.0)
Image: '/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince2/Images//20221014_1007_Plate04//Img2/Img_r01_c03.tif' Imp: 'null' Offset: (6552.0, 0.0)
Image: '/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince2/Images//20221014_1007_Plate04//Img2/Img_r01_c04.tif' Imp: 'null' Offset: (9828.0, 0.0)
Image: '/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince2/Images//20221014_1007_Plate04//Img2/Img_r01_c05.tif' Imp: 'null' Offset: (13104.0, 0.0)
Image: '/run/user/357100554/gvfs/smb-sha